<a href="https://colab.research.google.com/github/ashimasinghal/T5-for-Text-Generation/blob/main/Models/T5_BeamSearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
!ls '/content/gdrive/MyDrive/t5_sample_script.py'

/content/gdrive/MyDrive/t5_sample_script.py


In [ ]:
!pip install transformers==3.5.1

In [ ]:
!pip install torch==1.4.0

In [ ]:
import torch
from transformers import T5ForConditionalGeneration,AutoTokenizer
import pandas as pd


In [ ]:
% pip install sentencepiece

In [ ]:
def set_seed(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(42)

model = T5ForConditionalGeneration.from_pretrained('ramsrigouthamg/t5_paraphraser')
tokenizer = AutoTokenizer.from_pretrained('ramsrigouthamg/t5_paraphraser')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print ("device ",device)
model = model.to(device)


file=open(r'/content/gdrive/MyDrive/sample-text.txt',"r")
sentence_li=[]
while True:
    l=file.readline()
    if type(l)==str:
        sentence_li.append(l)
    if not l:
        break
file.close()
sentence_li=[w.strip("\n") for w in sentence_li]

device  cpu


In [ ]:


temporal_list=[]
for i in sentence_li:
    sentence=i
    sub_list=[]

    text =  "paraphrase: " + sentence + " </s>"


    max_len = 256
    # print(text)
    model = T5ForConditionalGeneration.from_pretrained('ramsrigouthamg/t5_paraphraser')
    tokenizer = AutoTokenizer.from_pretrained('ramsrigouthamg/t5_paraphraser')
    # print(tokenizer)

    # encoding = tokenizer.encode_plus(text,pad_to_max_length=True, return_tensors="pt")
    encoding = tokenizer.encode_plus(text,pad_to_max_length=True, return_tensors="pt")
    input_ids, attention_masks = encoding["input_ids"].to(device), encoding["attention_mask"].to(device)


    # set top_k = 50 and set top_p = 0.95 and num_return_sequences = 3
    beam_outputs = model.generate(
    input_ids=input_ids, attention_mask=attention_masks,
    do_sample=True,
    max_length=256,
    top_k=120,
    top_p=0.98,
    early_stopping=True,
    num_return_sequences=10
    )


    print ("\nOriginal Question ::")
    print (sentence)
    print ("\n")
    print ("Paraphrased Questions :: ")
    final_outputs =[]
    for beam_output in beam_outputs:
        sent = tokenizer.decode(beam_output, skip_special_tokens=True,clean_up_tokenization_spaces=True)
        if sent.lower() != sentence.lower() and sent not in final_outputs:
            final_outputs.append(sent)

    for i, final_output in enumerate(final_outputs):
        print("{}: {}".format(i, final_output))
        sub_list.append(final_output)
    temporal_list.append(sub_list)

sample_df=pd.DataFrame(columns=["Actual","Predicted"])
sample_df["Actual"]=sentence_li
sample_df["Predicted"]=temporal_list
sample_df.to_csv("/content/gdrive/MyDrive/paraphrase_t5.csv",index=False)
sample_df


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2022: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_t5.py:184: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."



Original Question ::
want to speak to parkway cancer centre medical director


Paraphrased Questions :: 
0: I am a cancer survivor, want to speak to Parkway Cancer Centre doctor. For more information on phone or to arrange for a free telephone consultation with him at the parkway cancer centre.
1: want to speak to parkway cancer centre medical director Dr Alex Jayne.
2: Can I speak to parkway cancer centre medical director in South Yorkshire?
3: Why would a cancer treatment centre contact steve j. parkway?
4: Would you like to speak to Stanbrook Cancer Centre medical director?
5: What is your view on parkway cancer centre's contact.
6: Need your question answered by Parkway Cancer Centre medical director?
7: I want to speak to parkway cancer centre medical director. want to talk to parkway cancer centre medical director?
8: I wish to speak to parkway cancer centre medical director and have many years experience. Want to speak to parkway cancer centre medical director?
9: would you lik